In [1]:
from nltk.corpus import PlaintextCorpusReader, stopwords
from nltk import WordPunctTokenizer, TreebankWordTokenizer, FreqDist, word_tokenize, WhitespaceTokenizer, MWETokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.util import ngrams
import csv

class InvertedIndex:
    """
    Construct Inverted Index
    """
    def __init__(self):
        # The dictionaries to hold the data of the inverted index
        self.inverted_index_docfreq = FreqDist()    # Store the inverted index term document frequency in a 
                                                    # frequency distribution from NLTK for easier use
        self.inverted_index_postings = dict(dict()) # A nested dictionary for words and then their appereance
                                                    # in each document by id {word:{docID:nr_of_apperanes}}
        self.token_sequence = dict(list())       # Dictionary with docIDs as keys and the tokens in correct sequence
                                                 # for proximity searches
        self.documents = {}                      # Dictionary with the indexes as keys and the documents as text
        
        # Lists to keep track of characters and locations
        self.characters = []
        self.locations = []                      
        
        # Multiple types of tokenizers
        self.tkn_p = WordPunctTokenizer()
        self.tkn_m = MWETokenizer(separator=' ')
        
        # Stopwords to be removes (from stop_words, nltk.corpus and manually inputed)
        stop_words = list(stopwords.words('english')) # Contains nt, m, s, d, etc...
        extra_words = ("'",',','"','.',']','.[','[','",','(','".[','."[',':','][',')',';','".',"—",'←','→','•',
                      '"[','),','.""[',',"', "’", "–", "’’", "''",)
        stop_words.extend(extra_words)
        self.stopwords = stop_words
        
        # Word processing tool (lemmatizing)
        self.lem = WordNetLemmatizer()
        
    
    def read_data(self, path: str) -> list:
        """
        Read files from a directory and then append the data of each file into a list.
        """
        files = PlaintextCorpusReader(path, ".*.txt")
        data_list = list()
        # This is for sorting the Simpsons episodes in order
        docIDs = []
        list_ids = [x.split('.') for x in files.fileids()]
        list_ids = sorted(list_ids, key=lambda x: (x[0],int(x[1])))
        for i in range(len(list_ids)):
            ex = ".".join(list_ids[i])
            docIDs.append(ex)
        # Put data from files in a nested list - first term is the id and second is the text
        for i in docIDs:
            fixed_raw = files.raw(i).replace("’", "'")  # Replace missleading "’" in data to "'" (apostrophe)
            fixed_raw = fixed_raw.replace("–", "-")     # Replace missleading "–" in data to "-" (hyphen)
            data_list.append([i[:-4], fixed_raw])
        
        locations_csv = self.read_csv(path, 'simpsons_locations.csv')   # List of lists for locations
        characters_csv = self.read_csv(path, 'simpsons_characters.csv') # List of lists for characters
        for i, v in enumerate(characters_csv):         # Correct simpsons_characters.csv "'s" normalization
            if "'s" in characters_csv[i][1]:
                characters_csv[i][2] = characters_csv[i][1].replace("'s", "").lower()
        
        # Fill the 2 lists (locations and characters) with data from the csv files
        # Tokenize data first and also add it to the MWEs for the MWETokenizer()
        for i, n, _ in locations_csv:
            item = self.tokenization(n, self.tkn_p, option="casefold+stopwords")
            self.tkn_m.add_mwe(item)
            self.locations.extend(self.tkn_m.tokenize(item))
        for i, n, _, _ in characters_csv:
            item = self.tokenization(n, self.tkn_p, option="casefold+stopwords")
            self.tkn_m.add_mwe(item)
            self.characters.extend(self.tkn_m.tokenize(item))
        
        return data_list

    def process_document(self, document: str) -> list:
        """
        Pre-process a document and return a list of its terms
        str->list
        """
        docID = document[0]
        raw_doc = document[1]
        # Update the inverted index documents dictionary with the raw data
        self.documents[docID] = raw_doc
        
        # The tokenization of the text string
        # 2 options: "casefold" or "casefold+stopwords"
        terms = self.tokenization(self.documents[docID], self.tkn_p, option="casefold+stopwords")
        # Applying the Multi-Word Expression to the tokens
        terms_mwe = self.tkn_m.tokenize(terms)
        
        return terms_mwe
    
    def index_corpus(self, documents: list) -> None:
        """
        Index given documents
        list->None
        """
        # Index given documents in the dictionaries of the inverted index
        for data in documents:
            docID = data[0]
            terms_final = self.process_document(data)
            # Update the document frequency
            self.inverted_index_docfreq.update(terms_final)
            self.token_sequence[docID] = list()
            for term in terms_final:
                # Add the term in the postings if it is not already there
                if term not in list(self.inverted_index_postings.keys()):
                    self.inverted_index_postings[term] = dict(dict())
                    for i in documents:
                        self.inverted_index_postings[term][i[0]] = 0
                # Update the inverted index postings
                self.inverted_index_postings[term][docID] += 1
                # Update the token sequence
                self.token_sequence[docID].append(term)
        
        return None
     
    def proximity_search(self, term1: str, term2: str, option="proxy") -> dict:
        """
        1) check whether given two terms appear within a window
        2) calculate the number of their co-existance in a document
        3) add the document id and the number of matches into a dict
        return the dict
        """
        dictionary = dict()
        window_size = 3 # How far to look to the left and right of a term for the "proxy" option
        ngram_size = 4  # Size of ngrams for the "ngrams" option
        # Do the proximity search by finding the term then making a list with its proximity
        if option=="proxy":
            for docID in self.token_sequence.keys():
                counter = 0
                for i, t in enumerate(self.token_sequence[docID]):
                    proximity = []
                    if t == term1:
                        for x in range(window_size):
                            proximity.append(self.token_sequence[docID][i-(x+1)])
                            proximity.append(self.token_sequence[docID][i+(x+1)])
                    if term2 in proximity:
                        counter += 1
                        dictionary[docID] = counter
        # Do the proximity search by making n-grams with the tokens and looking through them
        elif option=="ngrams":
            for docID in self.token_sequence.keys():
                counter = 0
                n_grams = ngrams(self.token_sequence[docID], ngram_size)
                for g in n_grams:
                    if term1 in g and term2 in g:
                        counter += 1
                        dictionary[docID] = counter
        return dictionary
        
        
    def tokenization(self, document, tkn, option="") -> list:
        """
        Tokenization method with multiple options
        of tokenization depending on the option input
        also contains lemmatization and stemming if need be
        """
        terms = tkn.tokenize(document)
        # Just casefolding
        if option == "casefold":
            terms_lower = [t.lower() for t in terms]
            terms = terms_lower
        # Casefolding and removing stopwords
        elif option == "casefold+stopwords":
            terms_stop = []
            temp_term = ""
            for t in terms:
                # Connect the words with a hyphen (-) in between them
                if t.lower() == "-" or (len(terms_stop) > 0 and terms_stop[-1][-1] == "-"):
                    if temp_term != "":
                        terms_stop.append(temp_term + "-")
                        temp_term = ""
                        continue
                    terms_stop[-1] = terms_stop[-1] + t.lower()
                    continue
                # If word not in stopwords then add it to final terms list
                elif t.lower() not in self.stopwords:
                    terms_stop.append(t.lower())
                    temp_term = ""
                # Make sure to also keep the tokens with a stopword but connected by a hyphen
                else:
                    temp_term = t.lower()
            terms = terms_stop
            
        # Lemmatizing
        terms_final = self.tokenize_lem_stm(terms, option="lem")
        
        return terms_final
    
    def tokenize_lem_stm(self, terms, lem=WordNetLemmatizer(), stm=SnowballStemmer("english"), option="") -> list:
        """
        Method for lemmatization or stemming or both
        """
        terms_final = []
        # Lemmatization
        if option == "lem":
            terms_final = [lem.lemmatize(t) for t in terms]
            return terms_final
        return terms
    
    def read_csv(self, path, file_name):
        """
        Method to read from a csv file
        """
        file = open(path + file_name)
        csvreader = csv.reader(file)
        next(csvreader)
        data = [row for row in csvreader]
        file.close()
        return data

In [2]:
def main():
    "main call function"
    char = False                          # bool to work around location and character names
    loc = False
    worked = False
    index = InvertedIndex()               # initilaise the index
    corpus = index.read_data('Simpsons/') # specify the directory path in which files are located
    index.index_corpus(corpus)            # index documents/corpus
    
    search_term = str()
    search_term_original = input("Enter your query: ") # insert a query
    
    # Tokenize search term for better search in the inverted index
    search_term_list = index.tokenization(search_term_original, index.tkn_p, option="casefold+stopwords")
    search_terms = index.tkn_m.tokenize(search_term_list)
    print("Tokenized for searching as: " + str(search_terms))
    
    # Checking if entered terms are characters or locations
    for n in index.characters:
        if n in search_terms:
            print("You entered a: Character")
            char = True
            break
    for n in index.locations:
        if n in search_terms:
            print("You entered a: Location")
            loc = True
            break
    # Search for 1 term
    if len(search_terms) == 1:
        search_term = search_terms[0]
        print("\nQUERY SEARCH: Appears as a term/token ("+search_term+") in:")
        
        # Calculate apperance of the term
        apperances = dict()
        for t in index.inverted_index_postings.keys():
            if t == search_term:
                for i in index.inverted_index_postings[t].keys():
                    if index.inverted_index_postings[t][i] != 0:
                        apperances[i] = index.inverted_index_postings[t][i]
        
        # Display apperance of the term
        print(str(len(apperances.keys())) + " episodes " + str(list(apperances.keys())))
        for k in apperances.keys():
            print(str(apperances[k]) + " times in episode: " + str(k))
        print(str(sum(apperances.values())) + " times in total")
        worked = True
    # Search proximity of 2 terms
    if len(search_terms) == 2 or ((char == True or loc == True) and len(search_term_list) == 2):
        # Check if terms are: (character + term) or (character name with 2 terms)
        if len(search_terms) == 2:
            print("\nPROXIMITY: Appears as 2 terms ("+str(search_terms[0])+") and ("+str(search_terms[1])+") in:")
            dictionary = index.proximity_search(search_terms[0], search_terms[1], option="proxy")
        else:
            print("\nPROXIMITY: Appears as 2 terms ("+str(search_term_list[0])+") and ("+str(search_term_list[1])+") in:")
            dictionary = index.proximity_search(search_term_list[0], search_term_list[1], option="proxy")
            
        # Display apperances of the terms
        print(str(len(dictionary.keys())) + " episodes " + str(list(dictionary.keys())))
        for k in dictionary.keys():
            print(str(dictionary[k]) + " times in episode: " + str(k))
        print(str(sum(dictionary.values())) + " times in total")
        worked = True
            
    if worked == False:
        print("Unrecognized term/terms or too many terms entered")
    return index
    
index = main()

Enter your query: Santa's Little Helper
Tokenized for searching as: ['santa little helper']
You entered a: Character

QUERY SEARCH: Appears as a term/token (santa little helper) in:
7 episodes ['3.2', '3.4', '3.11', '3.16', '3.19', '3.22', '4.5']
1 times in episode: 3.2
1 times in episode: 3.4
1 times in episode: 3.11
1 times in episode: 3.16
28 times in episode: 3.19
1 times in episode: 3.22
1 times in episode: 4.5
34 times in total
